In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np
from sklearn.datasets import load_iris 
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [3]:
iris = load_iris()
df = pd.DataFrame(iris.data, columns=iris.feature_names)
df['target'] = iris.target

In [4]:
df.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


In [5]:
train_df,test_df = train_test_split(df,test_size= 0.20,random_state = 42,stratify=df['target'])



In [7]:
x_train,y_train = train_df.drop(columns=['target']),train_df['target']
x_test,y_test= test_df.drop(columns=['target']),test_df['target']



In [8]:
scaler= StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test= scaler.transform(x_test)


In [10]:
x_train_tensor= torch.tensor(x_train,dtype= torch.float32)
x_test_tensor= torch.tensor(x_test,dtype= torch.float32)
y_train_tensor= torch.tensor(y_train.values,dtype= torch.long)
y_test_tensor= torch.tensor(y_test.values,dtype= torch.long)

In [20]:
class IrisClassifier(nn.Module):
    def __init__(self,input_dim,hidden_dim,output_dim):
        super(IrisClassifier,self).__init__()
        self.network = nn.Sequential(
            nn.Linear(input_dim,hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim,hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim,output_dim)
            
        )
        
    def forward(self,x):
        return self.network(x)
    
    
    

In [21]:
inuput = x_train.shape[1]
hidden = 16
out = 3


In [22]:
model = IrisClassifier(inuput,hidden,out)

In [23]:
cre = nn.CrossEntropyLoss()
optimizer= optim.Adam(model.parameters(),lr= 0.01)


In [26]:
epochs = 500
for e in range(epochs):
    model.train()
    optimizer.zero_grad()
    pred= model(x_train_tensor)
    loss= cre(pred,y_train_tensor)
    loss.backward()
    optimizer.step()
    
    if (e+1)%50 ==0:
        print(f"Epoch [{epochs+1}/{epochs}],loss :{loss.item():.4f}")

Epoch [501/500],loss :0.0304
Epoch [501/500],loss :0.0222
Epoch [501/500],loss :0.0116
Epoch [501/500],loss :0.0048
Epoch [501/500],loss :0.0023
Epoch [501/500],loss :0.0013
Epoch [501/500],loss :0.0009
Epoch [501/500],loss :0.0006
Epoch [501/500],loss :0.0005
Epoch [501/500],loss :0.0004


In [28]:
model.eval()
with torch.no_grad():
    y_pred =model(x_test_tensor)
    y_pred_lables = torch.argmax(y_pred,dim=1)
    
    accuracy = (y_pred_lables==y_test_tensor).sum().item()/y_test_tensor.size(0)
    
    print(f"Accuracy :{accuracy}")

Accuracy :0.9666666666666667


In [29]:
def pred_irsi(sep_le,sep_wid,pet_len,pet_wid):
    input_data= np.array([[sep_le,sep_wid,pet_len,pet_wid]])
    
    input_data = scaler.transform(input_data)
    input_tensor = torch.tensor(input_data,dtype= torch.float32)
    
    model.eval()
    with torch.no_grad():
        pred =model(x_test_tensor)
        pred_lables = torch.argmax(pred,dim=1)
    return iris.target_names[pred_lables]
    

In [30]:
g= pred_irsi(5.1,3.5,1.4,0.2)

c:\Users\saiki\anaconda3\lib\site-packages\sklearn\base.py:465: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [31]:
print(g)

['setosa' 'virginica' 'versicolor' 'versicolor' 'setosa' 'versicolor'
 'setosa' 'setosa' 'virginica' 'versicolor' 'virginica' 'virginica'
 'virginica' 'versicolor' 'setosa' 'setosa' 'setosa' 'versicolor'
 'versicolor' 'virginica' 'setosa' 'virginica' 'versicolor' 'virginica'
 'virginica' 'virginica' 'versicolor' 'setosa' 'virginica' 'setosa']
